In [1]:
import numpy as np
import numpy
import sklearn.cluster

In [2]:
import collections
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\schwe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\schwe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [99]:
sentences = []

with open('descriptions.txt', encoding = "utf8") as f:
    for line in f:
        text = line.lower()                                       ## Lowercase all characters
        text = text.replace("[comma]"," ")                        ## Replace [commas] with empty space
        for ch in text:
            if ch < "0" or (ch < "a" and ch > "9") or ch > "z":   ## The cleaning operation happens here, remove all special characters
                text = text.replace(ch," ")
        text = ' '.join(text.split())                             ## Remove double spacing from sentences
        sentences.append(text)
        
sentences = sentences[0:100]
nclusters = int(len(sentences)/5)

print(len(sentences), nclusters)


100 20


In [4]:
def word_tokenizer(text):
            #tokenizes and stems the text
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('english')]
    return tokens

In [196]:
def cluster_sentences(sentences, nb_of_clusters):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                    stop_words=stopwords.words('english'),
                                     max_df=0.9,
                                    min_df=0.1,
                                    lowercase=True)
            #builds a tf-idf matrix for the sentences
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    kmeans = KMeans(n_clusters=nb_of_clusters)
    kmeans.fit(tfidf_matrix)
    clusters = collections.defaultdict(list)
    for i, label in enumerate(kmeans.labels_):
            clusters[label].append(i)
    
    
    ##this code replicates value for when the cluster is too small
    ## it also truncates clusters that are too big
    ##but we may need to do something before this to replicate the items according to how many values are in it
    
    for key, item in clusters.items():
        if len(item) < 5:
            missing = 5 - len(item)
            for i in range(missing):
                val = item[1]
                clusters[key].append(val)
        if len(item) > 5:
            clusters[key] = item[0:5]

    return dict(clusters)

In [197]:
clusters = cluster_sentences(sentences, nclusters)

In [73]:
print(clusters[0])
print()
print(clusters[295])

[107, 123, 201, 1076]

[69]


[[25, 26, 27, 28, 29], [30, 32, 40, 41, 42, 44], [10, 11, 12, 13, 14], [17, 19, 24, 31, 33, 34, 43], [5, 6, 7, 8, 9], [15, 16, 18], [45, 46, 47, 48, 49], [0, 1, 2, 3, 4], [35, 36, 37, 38, 39], [20, 21, 22, 23]]


[[  0.   0.   0. ...   0.   0.   0.]
 [  1.   0.   0. ...   0.   0.   0.]
 [  2.   0.   0. ...   0.   0.   0.]
 ...
 [293.   0.   0. ...   0.   0.   0.]
 [294.   0.   0. ...   0.   0.   0.]
 [295.   0.   0. ...   0.   0.   0.]]
